## Diseño de filtros pasa-banda

### Diseño
Este script hace el diseño de un filtro FIR pasa bajos y lo convierte en un pasa banda con las mismas características del pasa bajo. Para hacer la conversión se multiplica por una exponencial centrada en la frecuencia requerida para el pasa banda.

Este script está basado en el script fir2.py de esta carpeta


Importo los modulos necesarios

In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sg, fftpack as fp, interpolate as ip
from dsp_funcs import remezord, lporder, h_freqz, mfreqz, impz, zplane

Especificaciones de diseño del filtro FIR pasa-bajas.

In [ ]:
Fs = 8e3       # Frecuencia de muestreo
Fc = 1.5e3     # Frecuencia central para el filtro PB. En este caso coincide con f1
a_p = 0.1      # Máxima atenueación en la banda de paso
a_s = 60       # Minima atenueación en la banda atenuada
f_p = 0.0025   # = 10/(Fs/2) --> 10Hz va a ser el ancho de mi filtro pasa banda después
f_s = 0.00375; # = 15/(Fs/2) --> 15Hz empieza la banda de atenuación 
delta_p = (10**(a_p/20)-1)/(10**(a_p/20)+1) # Passband and stopband ripples
delta_s = (1+delta_p)*10**(-a_s/20)         # Passband and stopband ripples
dev = [delta_p, delta_s]             

Diseño del filtro pasa-bajas

In [ ]:
F = [f_p,f_s]; # Cutoff frequencies
A = [1, 0]; # Desired amplitudes
(Nord,Fo,Ao,W) = remezord(F, A, dev)# Estimating the filter order
print(Nord, Fo, Ao, W)

Al orden estimado le sumo 4 para hacerlo un poco más holgado y luego, con eso, utilizo remez para diseñar el filtro pasa-bajas.

In [ ]:
Nord = Nord+4;
#h = firpm(Nord,Fo,Ao,W); 
lp_taps=sg.remez(Nord,np.array(Fo)/2,Ao[0::2],W)# Filter coefficients #[0, pb_freq/2, sb_freq/2, 0.5],[1,0],w)

Ahora diseño el filtro pasa-banda a partir de los coeficientes del filtro pasa-bajas. Para eso, 
multiplico los coeficientes del filtro pasa-basjas por una exponencial compleja.

In [ ]:
#w, h = sg.freqz(lp_taps, 1, worN=1024)
#lowpass_taps = remez(ntaps, [0,0.0025/2,0.00375/2,0.5], [1, 0], maxiter=2500)
#bandpass_taps = [lp_tap * cos(2*pi*F_c/Fs*n) for n, lptap in enumerate(lowpass_taps)]
bp_taps = [lptap * np.exp(2j*np.pi*(Fc/Fs)*n) for n, lptap in enumerate(lp_taps)]

Ahora empiezo las pruebas. Genero una señal de prueba para analizar con el filtro diseñado.

In [ ]:
duration = 1.0
samples = int(Fs*duration)
n = np.arange(samples)
f1 = 1.5e3 #señal que quiero recuperar
f2 = 190.0 #señal espúrea
f3 = 102.0 #señal espúrea
fc = 1.5e3 #frecuencia central para mi filtro PB. En este caso coincide con f1
           #porque es esa frecuencia la que estoy queriendo recuperar
sn = np.random.randn(n.size) #ruido normalmente distribuido
signal_test = (5.0*np.sin(2.0*np.pi*(f1/Fs)*n)) + (0.5*np.sin(2.0*np.pi*(f2/Fs)*n)) + (0.5*np.sin(2.0*np.pi*(f3/Fs)*n)) + sn

Grafico las señales.

In [ ]:
fig,ax=plt.subplots(5,1)
ax[0].plot(n,5.0*np.sin(2.0*np.pi*(f1/Fs)*n))
ax[1].plot(n,5.0*np.sin(2.0*np.pi*(f2/Fs)*n))
ax[2].plot(n,5.0*np.sin(2.0*np.pi*(f3/Fs)*n))
ax[3].plot(n,sn)
ax[4].plot(n,signal_test)
#plt.show()

Para ver las señales presentes, grafico el espectro de frecuencias.

In [ ]:
X=fp.fft(signal_test,2**13)
f=fp.fftfreq(2**13,d=1/Fs)
fig, ax = plt.subplots()
ax.plot(f[:2**12],(2/len(signal_test))*abs(X[:2**12]))
#plt.show()

Aplico el filtro a la señal de prueba y observo la salida.

In [ ]:
#signal_test_filtered = lfilter(taps, 1, signal_test)
signal_test_filtered = sg.lfilter(bp_taps, 1, signal_test)

In [ ]:
h_freqz(lp_taps)

#Este grafico para ver todo el espectro [0, 2*np.pi]
#w, h = sg.freqz(lp_taps, 1, worN=1024, whole=True)
#h_dB = 20 * np.log10(np.abs(h))
#h_Phase = np.unwrap(np.arctan2(np.imag(h),np.real(h)))
#fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 7))
#ax[0].plot(w*0.5*Fs/np.pi,h_dB)
#ax[0].grid()
#ax[0].set_ylim(-90, 5)
##ax[0].set_xlim(0, 1)
#ax[0].set_ylabel('Magnitude (dB)')
##ax[0].set_xlabel(r'Normalized Frequency (x $\pi$ rad/sample)')
#ax[0].set_title(r'Frequency response')
#ax[1].plot(w*0.5*Fs/np.pi,(180/np.pi)*h_Phase)
#ax[1].grid()
##ax[1].set_xlim(0, 1)
##ax[1].ylabel('Phase (radians)')
#ax[1].set_ylabel('Phase (degrees)')
#ax[1].set_xlabel(r'Normalized Frequency (x $\pi$ rad/sample)')

In [ ]:
#plt.figure()
h_freqz(bp_taps)

#Este grafico para ver todo el espectro [0, 2*np.pi]
#w, h = sg.freqz(bp_taps, 1, worN=1024)
#h_dB = 20 * np.log10(np.abs(h))
#h_Phase = np.unwrap(np.arctan2(np.imag(h),np.real(h)))
#fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 7))
#ax[0].plot(w*0.5*Fs/np.pi,h_dB)
#ax[0].grid()
#ax[0].set_ylim(-90, 5)
##ax[0].set_xlim(0, 1)
#ax[0].set_ylabel('Magnitude (dB)')
##ax[0].set_xlabel(r'Normalized Frequency (x $\pi$ rad/sample)')
#ax[0].set_title(r'Frequency response')
#ax[1].plot(w*0.5*Fs/np.pi,(180/np.pi)*h_Phase)
#ax[1].grid()
#ax[1].set_xlim(0, 1)
#ax[1].ylabel('Phase (radians)')
#ax[1].set_ylabel('Phase (degrees)')
#ax[1].set_xlabel(r'Normalized Frequency (x $\pi$ rad/sample)')

Ahora grafico la respuesta.

In [ ]:
fig, ax = plt.subplots()
ax.plot(n, signal_test, label='signal_test')
ax.plot(n, signal_test_filtered.real, label='signal_test_filtered')
ax.set_xlabel("Time index")
ax.set_ylabel("Amplitude (V)")
ax.legend()
#fig.show()